# Polygons
---

Unstructured Grids are often used in the Dynamical Cores of climate models because of their ability to represent complex geometries effectively. However, the way unstructured grids are represented makes it difficult to produce quick visualizations.

Previously, we have discussed that data variables are typically mapped to the elements that compose unstructured grids (nodes, edges, and faces).When data is mapped to the faces, each face is shaded with the value of the data variable. We can treat these faces as Polygons on a plane for visualization.

This notebook showcases how UXarray can visualization data as polygons.

## Setup

In [ ]:
import uxarray as ux

In [ ]:
fig_size = 400
plot_kwargs = {"backend": "matplotlib", "aspect": 2, "fig_size": fig_size}

In [ ]:
file_dir = "../../meshfiles/"
grid_filename_mpas = file_dir + "oQU480.grid.nc"
data_filename_mpas = file_dir + "oQU480.data.nc"
uxds = ux.open_dataset(grid_filename_mpas, data_filename_mpas)

## Vector Polygon Plots

The `UxDataArray.plot.polygons()` method produces a Vector Polygon plot, with each face represented as a polygon and rendered exactly as it is mathematically represented.



In [ ]:
uxds["bottomDepth"].plot.polygons(**plot_kwargs)

### Interactive Example

In the plot below, take some time to zoom and pan around the plot to observe some of the defining characteristics of a Vector Polygon Plot:
* Hovering over a polygon (face) displays the value mapped to it, which is the same as the original value stored in our `UxDataArray`
* There is no loss of data fidelity when zooming in
* Each polygon is bounded by edges that are drawn as lines



In [ ]:
uxds["bottomDepth"].plot.polygons(
    backend="bokeh", width=1000, height=500, tools=["hover"]
)

When we zoom into our grid, we can observe the high-level of data fidelity that these plots produce.

In [ ]:
uxds["bottomDepth"].plot.polygons(xlim=(-20, 0), ylim=(-5, 5), **plot_kwargs)

## Rasterized Polygon Plots

The `UxDataArray.plot.rasterize(method='polygon')` function produces a Rasterized Polygon plot. Instead of rendering each polygon exactly, they are rasterized into cells using Datashader internally. 

In [ ]:
uxds["bottomDepth"].plot.rasterize(method="polygon", **plot_kwargs)

### Interactive Example

In the plot below, take some time to zoom and pan around to observe some of the defining characters of a Raster Polygon Plot and compare it to the Vector Polygon Plot
* Zooming in exposes the fact that each polygon is approximately rendered, with jagged edges around the boundaries of each cell
* Hovering over a polygon (face) still displays the original data values, even though the boundaries are not explicit like the vector example


In [ ]:
uxds["bottomDepth"].plot.rasterize(
    method="polygon", backend="bokeh", width=1000, height=500, tools=["hover"]
)

Zooming in here exposes that Raster Polygon Plots are simply approximations of the original geometries that are rasterized onto a fixed-size grid. Compared to the Vector Polygon Plot, the effective resolution is much lower. 

In [ ]:
uxds["bottomDepth"].plot.rasterize(
    method="polygon", xlim=(-20, 0), ylim=(-5, 5), **plot_kwargs
)

However, a higher level of data fidelity can be achieved by specifying a `pixel_ratio`, which controls the size of the binning used for rasterization. 
* A high pixel ratio increases the number of bins by making each bin smaller, leading to a higher effective resolution and a better approximation of the vector polygon plot
* A low pixel ratio decreases the number of bins by making each bin larger, leading to a lower effective resolution with jagged and coarse approximations of each polygon

In [ ]:
(
    uxds["bottomDepth"].plot.rasterize(
        xlim=(-20, 0),
        ylim=(-5, 5),
        pixel_ratio=0.5,
        title="0.5 (low) Pixel Ratio",
        **plot_kwargs
    )
    + uxds["bottomDepth"].plot.rasterize(
        xlim=(-20, 0),
        ylim=(-5, 5),
        pixel_ratio=8.0,
        title="8.0 (high) Pixel Ratio",
        **plot_kwargs
    )
).opts(fig_size=fig_size / 1.5).cols(1)

## Polygons Around Antimeridian

When attempting to visualize unstructured meshes that reside on a sphere, it's necessary to consider the behavior of geometric elements near or on the Antimeridian. Elements that exist on or cross the antimeridian need to be corrected to properly visualize them. UXarray uses the antimeridian package to split faces along the antimeridian. More details can be found in their [documentation](https://antimeridian.readthedocs.io/en/stable/).

<!-- Using text-align on a parent div -->
<div style="text-align: center;">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/8d/Earth_map_with_180th_meridian.jpg/640px-Earth_map_with_180th_meridian.jpg" alt="Antimeridian Example" width="800"/>
</div>

### Opting to Include Antimeridian Polygons

To include and correct antimeridian polygons, we can set `exclude_antimeridian=False`. 

The following plots are zoomed in to a section along the antimeridian. We can see that our plot is split exactly across the antimeridian.

In [ ]:
(
    uxds["bottomDepth"].plot.polygons(
        xlim=(-185, -175),
        ylim=(-5, 5),
        exclude_antimeridian=False,
        title="Left of Antimeridian",
        **plot_kwargs
    )
    + uxds["bottomDepth"].plot.polygons(
        xlim=(175, 185),
        ylim=(-5, 5),
        exclude_antimeridian=False,
        title="Right of Antimeridian",
        **plot_kwargs
    )
).opts(fig_size=fig_size / 1.5).cols(1)

### Opting to Exclude Antimeridian Polygons

To exclude antimeridian polygons, we can set `exclude_antimeridian=True`. 

In the following plots, we can see that the polygons that were corrected above are now missing. Since there is a relatively high overhead involved in detecting and correcting each antimeridian polygon, this option excludes them from the plot. 

In [ ]:
(
    uxds["bottomDepth"].plot.polygons(
        xlim=(-185, -175),
        ylim=(-5, 5),
        exclude_antimeridian=True,
        title="Left of Antimeridian",
        **plot_kwargs
    )
    + uxds["bottomDepth"].plot.polygons(
        xlim=(175, 185),
        ylim=(-5, 5),
        exclude_antimeridian=True,
        title="Right of Antimeridian",
        **plot_kwargs
    )
).opts(fig_size=fig_size / 1.5).cols(1)

## Projection Support

As of the most recent UXarray release, geographic projections and transformations are not supported by UXarray's polygon plotting functionality directly. However, we can utilizing UXarray paired with HoloViz tools to generate these types of plots with a bit of extra effort.

In [ ]:
import cartopy.crs as ccrs
import holoviews as hv
import hvplot.pandas

hv.extension("matplotlib")

### Conversion to a SpatialPandas GeoDataFrame

In order to support visualization with the popular HoloViz stack of libraries (hvPlot, HoloViews, Datashader, etc.), UXarray provides methods for converting Grid and UxDataArray objects into a SpatialPandas GeoDataFrame, which can be used for visualizing the polygons that represent each grid, in addition to data variables.

In [ ]:
gdf = uxds["bottomDepth"].to_geodataframe()
gdf

<div class="admonition alert alert-info">
    <p class="admonition-title" style="font-weight:bold">See also:</p>
    For more infromation about UXarray and the conversion to a GeoDataFrame, please refer to the
    <a href="https://uxarray--762.org.readthedocs.build/en/762/examples/005-to-geodataframe-for-holoviz.html">UXarray Usage Examples </a>
</div>

### Using `hvPlot`

Once we have a `spatialpandas.GeoDataFrame`, we can use the `.hvplot` accessor to generate Polygon plots with projections and geographic features. Below are a few examples:

In [ ]:
projection = ccrs.Robinson()

gdf.hvplot.polygons(
    rasterize=True, c="bottomDepth", cmap="Blues", projection=projection, height=600
)

In [ ]:
projection = ccrs.Robinson(central_longitude=-180)

gdf.hvplot.polygons(
    rasterize=True, c="bottomDepth", cmap="Blues", projection=projection, height=600
)

In [ ]:
import geoviews.feature as gf

projection = ccrs.Robinson(central_longitude=-180)

gdf.hvplot.polygons(
    rasterize=True, c="bottomDepth", cmap="Blues", projection=projection, height=600
) * gf.coastline() * gf.borders()

In [ ]:
projection = ccrs.Orthographic(central_latitude=90)

gdf.hvplot.polygons(
    rasterize=True, c="bottomDepth", cmap="Blues", projection=projection, height=400
) * gf.coastline() * gf.borders()

## Takeaways

Polygon plotting is UXarray's flagship visualuzation method, which provides a high level of data fidelity by utilizing the unstructured grid's connectivity information.


**When should I use vector or raster plots?**
* Rasterization should be used for most applications, especially when working with moderate to large grids. Also consider using an appropriate pixel ratio value to get a higher effective resolution
* Vector plots should be used when working with a small grid, typically under 10,000 elements because of the high overhead needed to render each polygon directly

**When should I set `exclude_antimeridian=False?`**
* It is suggested to almost always keep `exclude_antimeridian=True` (the default option) since it leads to a very large perfromance increase
* For small grids or when excluding the polygons would lead to artifacts, it is reccomend to set it to False
